In [1]:
# Parameters
execution_date = "2023-05-26T00:00:00+00:00"


In [2]:
from delta import configure_spark_with_delta_pip
from delta.tables import DeltaTable
from pyspark.sql import SparkSession

In [3]:
builder = SparkSession.builder.master("spark://spark-master:7077").appName("rawApp") \
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog")

spark = configure_spark_with_delta_pip(builder).getOrCreate()

/home/airflow/.local/lib/python3.7/site-packages/pyspark/bin/load-spark-env.sh: line 68: ps: command not found


:: loading settings :: url = jar:file:/home/airflow/.local/lib/python3.7/site-packages/pyspark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/airflow/.ivy2/cache
The jars for the packages stored in: /home/airflow/.ivy2/jars
io.delta#delta-core_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-8a1fa794-ebec-4640-9fc9-32ddd1bbf30c;1.0
	confs: [default]


	found io.delta#delta-core_2.12;2.2.0 in central


	found io.delta#delta-storage;2.2.0 in central


...

	[SUCCESSFUL ] io.delta#delta-core_2.12;2.2.0!delta-core_2.12.jar (1271ms)


downloading https://repo1.maven.org/maven2/io/delta/delta-storage/2.2.0/delta-storage-2.2.0.jar ...
	[SUCCESSFUL ] io.delta#delta-storage;2.2.0!delta-storage.jar (623ms)


downloading https://repo1.maven.org/maven2/org/antlr/antlr4-runtime/4.8/antlr4-runtime-4.8.jar ...
	[SUCCESSFUL ] org.antlr#antlr4-runtime;4.8!antlr4-runtime.jar (328ms)
:: resolution report :: resolve 4173ms :: artifacts dl 2226ms
	:: modules in use:
	io.delta#delta-core_2.12;2.2.0 from central in [default]
	io.delta#delta-storage;2.2.0 from central in [default]
	org.antlr#antlr4-runtime;4.8 from central in [default]
	---------------------------------------------------------------------
	|                  |            modules            ||   artifacts   |
	|       conf       | number| search|dwnlded|evicted|| number|dwnlded|
	---------------------------------------------------------------------
	|      default     |   3   |   3   |   3   |   0   ||   3   |   3   |
	---------------------------------------------------------------------
:: retrieving :: org.apache.spark#spark-submit-parent-8a1fa794-ebec-4640-9fc9-32ddd1bbf30c
	confs: [default]
	3 artifacts copied, 0 already retrieved (3

23/05/27 04:05:56 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [4]:
df = spark.read.csv("/opt/workspace/data/input/Iris.csv", header=True)

In [5]:
df.write.mode("overwrite").format("delta").save("/opt/workspace/data/raw/iris/")

23/05/27 04:06:03 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


In [6]:
delta_table = DeltaTable.forPath(spark, "/opt/workspace/data/raw/iris/")

In [7]:
delta_table.history().show()

+-------+--------------------+------+--------+---------+--------------------+----+--------+---------+-----------+--------------+-------------+--------------------+------------+--------------------+
|version|           timestamp|userId|userName|operation| operationParameters| job|notebook|clusterId|readVersion|isolationLevel|isBlindAppend|    operationMetrics|userMetadata|          engineInfo|
+-------+--------------------+------+--------+---------+--------------------+----+--------+---------+-----------+--------------+-------------+--------------------+------------+--------------------+
|      3|2023-05-27 04:06:...|  null|    null|    WRITE|{mode -> Overwrit...|null|    null|     null|          2|  Serializable|        false|{numFiles -> 1, n...|        null|Apache-Spark/3.3....|
|      2|2023-04-04 20:26:...|  null|    null|    WRITE|{mode -> Overwrit...|null|    null|     null|          1|  Serializable|        false|{numFiles -> 1, n...|        null|Apache-Spark/3.3....|
|      1|2

In [8]:
spark.stop()